In [1]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dataset12 = pd.read_excel("ConsolidatedFiles/2020_1.xlsx")

In [4]:
dataset12

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,I wanted to get more hands on experience in ta...,More Excel classes and maybe see if we can get...,The classes i took that we used excel in helpe...,"Excel skills, along with basic tax knowledge.",I want to focus on the different tax rules and...,more knowledge on tax regulations.,Being in person at the office.,Asks good questions and positive attitude,na,na
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,Ever since I began pursuing a career in accoun...,I have no suggestions,I was able to apply what I learned in my accou...,All of my major accounting courses,I want to focus on excelling in my courses and...,I think I need to simply gain more knowledge i...,This job would have been much more interesting...,NaN,NaN,NaN
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,This co-op wasn't as beneficial as my first co...,NaN,I had to lead an Agile project and work with a...,Domestic and Global Outsourcing (MIS 347),NaN,Develop attainable goals and reasonable steps ...,The ability to interact with colleagues in office,Numbers based analytics / advanced excel,Self motivation (I believe this was a tough ye...,NaN
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,I learned how to troubleshoot Amazon's network...,NaN,"I learned on the job, and my desire to underst...",My coursework in MIS.,I want to focus on my MIS degree to pursue a c...,I need to develop my technical skills.,"If I had more female co-workers, the job would...",Christina is organized and focused. She accept...,There were times that she failed to get inform...,Just continue on the path of learning and take...
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,An aspect of this co-op experience that relate...,More accounting projects in classes that use M...,The financial accounting class I took at Drexe...,The coursework that prepared me for this co-op...,I hope to focus more on finance work in my ret...,Time Management_x000D_\nSetting goals and eval...,An onboarding process that included a comprehe...,Justin was able to pick up processes quickly a...,Justin has opportunity to develop his skills a...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,5562,Spring/Summer,B,MKTG,5COP,JR,International,First,This was my very first experience working at a...,I think it's better if we had more time for ou...,Helped with my resume.,"business, marketing and BSAN","I want to keep my grade in a good stage, and f...","excel, SQL and still quick learner","if I have more time for this, I really like th...",Nhi is very effective at analyzing data using ...,Nhi will need to continue to work on being mor...,None
1379,5912,Spring/Summer,B,FIN,5COP,PJ,Domestic,First,I have always cared about making sure there ar...,Make sure to inform students that clients may ...,Business communication projects forced me to a...,Mostly business communication due to the amoun...,I want to focus more on interpersonal communic...,I want to focus more on communication as well ...,Being able to see the housing process during t...,"Willie is polite and always willing to listen,...",Willie could be a little more assertive,"No changes recommended, our industry does not ..."
1380,5572,Spring/Summer,B,ACCT,5COP,JR,International,First,One of my professional goals is to become a CP...,"Making it more practical. For example, in acco...",I was prepared more by extracurricular activit...,Financial Reporting I & II,I'd like to take finance classes in addition t...,Procrastination and working at a better speed,Nothing that I can think of,Munira has been a fantastic asset to our Finan...,Munira is a great worker and therefore she has...,Introduce more real-life situational accountin...
1381,

In [5]:
sheetNames = dataset12.columns[8:]
columnList = list(dataset12.columns)
for i in range(8, len(columnList)):
    dataset12.rename({columnList[i]: sheetNames[i-8]}, axis=1, inplace=True)

In [6]:
datasetPP_1 = dataset12.copy()
# sheetNames.append('Skills_used')
datasetPP_1['Skills_used'] = datasetPP_1['EMP-Strengths'] + datasetPP_1['EMP-Weaknesses']
datasetPP_1.head(50)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,I wanted to get more hands on experience in ta...,More Excel classes and maybe see if we can get...,The classes i took that we used excel in helpe...,"Excel skills, along with basic tax knowledge.",I want to focus on the different tax rules and...,more knowledge on tax regulations.,Being in person at the office.,Asks good questions and positive attitude,na,na,Asks good questions and positive attitudena
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,Ever since I began pursuing a career in accoun...,I have no suggestions,I was able to apply what I learned in my accou...,All of my major accounting courses,I want to focus on excelling in my courses and...,I think I need to simply gain more knowledge i...,This job would have been much more interesting...,NaN,NaN,NaN,NaN
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,This co-op wasn't as beneficial as my first co...,NaN,I had to lead an Agile project and work with a...,Domestic and Global Outsourcing (MIS 347),NaN,Develop attainable goals and reasonable steps ...,The ability to interact with colleagues in office,Numbers based analytics / advanced excel,Self motivation (I believe this was a tough ye...,NaN,Numbers based analytics / advanced excelSelf m...
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,I learned how to troubleshoot Amazon's network...,NaN,"I learned on the job, and my desire to underst...",My coursework in MIS.,I want to focus on my MIS degree to pursue a c...,I need to develop my technical skills.,"If I had more female co-workers, the job would...",Christina is organized and focused. She accept...,There were times that she failed to get inform...,Just continue on the path of learning and take...,Christina is organized and focused. She accept...
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,An aspect of this co-op experience that relate...,More accounting projects in classes that use M...,The financial accounting class I took at Drexe...,The coursework that prepared me for this co-op...,I hope to focus more on finance work in my ret...,Time Management_x000D_\nSetting goals and eval...,An onboarding process that included a comprehe...,Justin was able to pick up processes quickly a...,Justin has opportunity to develop his skills a...,NaN,Justin was able to pick up processes quickly a...
5,79,Fall/Winter,B,FIN,5COP,SR,Domestic,Third/Final,One professional goal that I am pursuing is to...,Have more classes like COM270 or MGMT450 to he...,Business communication prepared me for my co-o...,"I applied COM270, Business communication most ...","Last co-op, won't be returning to classes. Gra...",Data analytics,This job would have been more interesting if I...,NaN,NaN,NaN,NaN
6,2367,Spring/Summer,B,MKTG,5COP,SR,Domestic,Third/Final,"I have always been interested in events, and a...",n/a _x000D_\nIt's difficult to learn in the cl...,I was prepared for Co-Op from my previous 2 Co...,MKTG 367 - data-driven marketing_x000D_\nWhen ...,I added an IDM minor so I am excited to learn ...,Continuing to set professional boundaries.,"Being paid, not doing administrative tasks lik...","Alessandra had a wonderful attitude, was highl...",Alessandra was wonderful and an asset to our t...,None,"Alessandra had a wonderful attitude, was highl..."
7,2371,Spring/Summer,B,MKTG,5COP,SR,Domestic,Third/Final,"From the beginning of my time at Macquarie, I ...",Frame coursework like deliverables.,The higher-level classes I have taken prepared...,"Digital Data-Driven Marketing, Corporate Socia...",I would like to hone my skills in the areas I ...,I need to get more comfortable speaking in mee...,I would've like to have a deeper connection wi...,NaN,NaN,NaN,NaN
8,2374,Spring/Summer,B,MIS

In [8]:
def pre_process(text):
    text = str(text)
    new_words = []
    stop_words = []
    stop_words = set(stopwords.words('english'))
    
    with open("includeWordList.txt", 'r') as file:
        inclusiveWords = [line.strip() for line in file.readlines()]
    
#     text = includeWordsProcess(text, inclusiveWords)
    for substring in inclusiveWords:
        if len(substring.split()) == 2:
            replacement = '_'.join(substring.split())
            text = text.replace(substring, replacement)
#     return context
    
    ##Creating a list of custom stopwords from a file
    with open('stopWordList.txt') as f:
        new_words=[word for line in f for word in line.split()]
#     new_words = flat
    stop_words = list(stop_words.union(new_words))
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string - Tokenization
    text = text.split()
    
    # remove stopwords
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]
    
    #Stemming
#     for word in text:
#         print ("{0:20}{1:20}".format(word, ps.stem(word)))
    
    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    
#     return text    # returns list in the dataframe ['an', 'apple', 'a', 'day']
    strx = ' '.join(text) # returns list in the dataframe as a sentence/string (words are joined by empty space character) 'an apple a day'
    from keybert import KeyBERT
    kw_model = KeyBERT()
    # print("\nKeyBert Result\n")
    keywords = kw_model.extract_keywords(strx)
        #KeyBert Result
    kwResult = kw_model.extract_keywords(strx, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
#     res = str(kwResult).strip('[]')
    return kwResult




for i in range (0, len(sheetNames)):
    print(sheetNames[i])
    datasetPP_1[sheetNames[i]] = datasetPP_1[sheetNames[i]].apply(pre_process)

Goals
Classroom_activities_improve
Classroom_activities
Coursework
Return_to_class
Skills_to_develop
Job_interest
EMP-Strengths
EMP-Weaknesses
EMP-Feedback


In [9]:
datasetPP_1['Year'] = datasetPP_1['Work_Terms'].str.slice(0, 4) 
datasetPP_1.head(5)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used,Year
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,"[(senior, 0.3388), (planning, 0.3095), (return...","[(excel, 0.5205), (access, 0.5079), (class, 0....","[(excel, 0.4781), (flow, 0.3821), (class, 0.33...","[(excel, 0.6459), (skill, 0.5102), (along, 0.2...","[(cpa, 0.5948), (regulation, 0.5108), (exam, 0...","[(regulation, 1.0)]","[(person, 1.0)]","[(attitude, 0.5697), (asks, 0.5537), (question...",[],[],Asks good questions and positive attitudena,Spri
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,"[(accounting, 0.4684), (achieve, 0.3884), (pur...","[(suggestion, 1.0)]","[(accounting, 0.8058), (course, 0.2962)]","[(accounting, 0.8058), (course, 0.2962)]","[(excelling, 0.5827), (assignment, 0.4584), (a...","[(audit, 0.5346), (accounting, 0.5313), (emplo...","[(remotely, 0.4959), (control, 0.3852), (inter...","[(nan, 1.0)]","[(nan, 1.0)]","[(nan, 1.0)]",NaN,Fall
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,"[(consulting, 0.524), (agile, 0.4505), (remote...","[(nan, 1.0)]","[(agile, 0.4656), (collaborate, 0.4461), (lead...","[(outsourcing, 0.6944), (mi, 0.4187), (global,...","[(nan, 1.0)]","[(attainable, 0.5867), (goal, 0.5282), (reach,...","[(colleague, 0.6858), (interact, 0.6406)]","[(excel, 0.6599), (number, 0.4163), (advanced,...","[(motivation, 0.6571), (problem_solving, 0.493...","[(nan, 1.0)]",Numbers based analytics / advanced excelSelf m...,Fall
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,"[(cybersecurity, 0.4593), (vulnerability, 0.34...","[(nan, 1.0)]","[(experience, 0.4807), (hand, 0.4766), (desire...","[(mi, 1.0)]","[(mi, 0.5272), (degree, 0.4376), (graduation, ...","[(technical_skills, 0.9151), (need, 0.2497)]","[(worker, 0.6074), (female, 0.5425), (interest...","[(christina, 0.453), (personality, 0.4148), (t...","[(failed, 0.416), (key, 0.408), (growth, 0.334...","[(certification, 0.5955), (path, 0.5071), (fie...",Christina is organized and focused. She accept...,Spri
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,"[(networking, 0.486), (network, 0.4726), (meet...","[(excel, 0.3904), (accounting, 0.3644), (hour,...","[(accounting, 0.7399), (finance, 0.5188), (boo...","[(pivottables, 0.6948), (excel, 0.4705), (form...","[(accounting, 0.5042), (finance, 0.4552), (cla...","[(management_x, 0.4565), (progress, 0.416), (g...","[(onboarding, 0.5589), (email, 0.3812), (overv...","[(justin, 0.509), (communicates, 0.4013), (pic...","[(internship, 0.5264), (delay, 0.3652), (justi...","[(nan, 1.0)]",Justin was able to pick up processes quickly a...,Spri


In [10]:
datasetPP_1['Skills_used'] = datasetPP_1['Skills_used'].apply(pre_process)
datasetPP_1.head(5)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used,Year
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,"[(senior, 0.3388), (planning, 0.3095), (return...","[(excel, 0.5205), (access, 0.5079), (class, 0....","[(excel, 0.4781), (flow, 0.3821), (class, 0.33...","[(excel, 0.6459), (skill, 0.5102), (along, 0.2...","[(cpa, 0.5948), (regulation, 0.5108), (exam, 0...","[(regulation, 1.0)]","[(person, 1.0)]","[(attitude, 0.5697), (asks, 0.5537), (question...",[],[],"[(attitudena, 0.6682), (asks, 0.5577), (questi...",Spri
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,"[(accounting, 0.4684), (achieve, 0.3884), (pur...","[(suggestion, 1.0)]","[(accounting, 0.8058), (course, 0.2962)]","[(accounting, 0.8058), (course, 0.2962)]","[(excelling, 0.5827), (assignment, 0.4584), (a...","[(audit, 0.5346), (accounting, 0.5313), (emplo...","[(remotely, 0.4959), (control, 0.3852), (inter...","[(nan, 1.0)]","[(nan, 1.0)]","[(nan, 1.0)]","[(nan, 1.0)]",Fall
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,"[(consulting, 0.524), (agile, 0.4505), (remote...","[(nan, 1.0)]","[(agile, 0.4656), (collaborate, 0.4461), (lead...","[(outsourcing, 0.6944), (mi, 0.4187), (global,...","[(nan, 1.0)]","[(attainable, 0.5867), (goal, 0.5282), (reach,...","[(colleague, 0.6858), (interact, 0.6406)]","[(excel, 0.6599), (number, 0.4163), (advanced,...","[(motivation, 0.6571), (problem_solving, 0.493...","[(nan, 1.0)]","[(motivation, 0.5161), (problem_solving, 0.514...",Fall
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,"[(cybersecurity, 0.4593), (vulnerability, 0.34...","[(nan, 1.0)]","[(experience, 0.4807), (hand, 0.4766), (desire...","[(mi, 1.0)]","[(mi, 0.5272), (degree, 0.4376), (graduation, ...","[(technical_skills, 0.9151), (need, 0.2497)]","[(worker, 0.6074), (female, 0.5425), (interest...","[(christina, 0.453), (personality, 0.4148), (t...","[(failed, 0.416), (key, 0.408), (growth, 0.334...","[(certification, 0.5955), (path, 0.5071), (fie...","[(personality, 0.4256), (aggressiveness, 0.364...",Spri
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,"[(networking, 0.486), (network, 0.4726), (meet...","[(excel, 0.3904), (accounting, 0.3644), (hour,...","[(accounting, 0.7399), (finance, 0.5188), (boo...","[(pivottables, 0.6948), (excel, 0.4705), (form...","[(accounting, 0.5042), (finance, 0.4552), (cla...","[(management_x, 0.4565), (progress, 0.416), (g...","[(onboarding, 0.5589), (email, 0.3812), (overv...","[(justin, 0.509), (communicates, 0.4013), (pic...","[(internship, 0.5264), (delay, 0.3652), (justi...","[(nan, 1.0)]","[(justin, 0.4682), (internship, 0.4404), (proc...",Spri


In [12]:
excel_filename = '2020PP.xlsx'
datasetPP_1.to_excel(excel_filename, index=False)
print(excel_filename)
datasetPP_1

2020PP.xlsx


,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used,Year
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,"[(senior, 0.3388), (planning, 0.3095), (return...","[(excel, 0.5205), (access, 0.5079), (class, 0....","[(excel, 0.4781), (flow, 0.3821), (class, 0.33...","[(excel, 0.6459), (skill, 0.5102), (along, 0.2...","[(cpa, 0.5948), (regulation, 0.5108), (exam, 0...","[(regulation, 1.0)]","[(person, 1.0)]","[(attitude, 0.5697), (asks, 0.5537), (question...",[],[],"[(attitudena, 0.6682), (asks, 0.5577), (questi...",Spri
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,"[(accounting, 0.4684), (achieve, 0.3884), (pur...","[(suggestion, 1.0)]","[(accounting, 0.8058), (course, 0.2962)]","[(accounting, 0.8058), (course, 0.2962)]","[(excelling, 0.5827), (assignment, 0.4584), (a...","[(audit, 0.5346), (accounting, 0.5313), (emplo...","[(remotely, 0.4959), (control, 0.3852), (inter...","[(nan, 1.0)]","[(nan, 1.0)]","[(nan, 1.0)]","[(nan, 1.0)]",Fall
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,"[(consulting, 0.524), (agile, 0.4505), (remote...","[(nan, 1.0)]","[(agile, 0.4656), (collaborate, 0.4461), (lead...","[(outsourcing, 0.6944), (mi, 0.4187), (global,...","[(nan, 1.0)]","[(attainable, 0.5867), (goal, 0.5282), (reach,...","[(colleague, 0.6858), (interact, 0.6406)]","[(excel, 0.6599), (number, 0.4163), (advanced,...","[(motivation, 0.6571), (problem_solving, 0.493...","[(nan, 1.0)]","[(motivation, 0.5161), (problem_solving, 0.514...",Fall
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,"[(cybersecurity, 0.4593), (vulnerability, 0.34...","[(nan, 1.0)]","[(experience, 0.4807), (hand, 0.4766), (desire...","[(mi, 1.0)]","[(mi, 0.5272), (degree, 0.4376), (graduation, ...","[(technical_skills, 0.9151), (need, 0.2497)]","[(worker, 0.6074), (female, 0.5425), (interest...","[(christina, 0.453), (personality, 0.4148), (t...","[(failed, 0.416), (key, 0.408), (growth, 0.334...","[(certification, 0.5955), (path, 0.5071), (fie...","[(personality, 0.4256), (aggressiveness, 0.364...",Spri
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,"[(networking, 0.486), (network, 0.4726), (meet...","[(excel, 0.3904), (accounting, 0.3644), (hour,...","[(accounting, 0.7399), (finance, 0.5188), (boo...","[(pivottables, 0.6948), (excel, 0.4705), (form...","[(accounting, 0.5042), (finance, 0.4552), (cla...","[(management_x, 0.4565), (progress, 0.416), (g...","[(onboarding, 0.5589), (email, 0.3812), (overv...","[(justin, 0.509), (communicates, 0.4013), (pic...","[(internship, 0.5264), (delay, 0.3652), (justi...","[(nan, 1.0)]","[(justin, 0.4682), (internship, 0.4404), (proc...",Spri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,5562,Spring/Summer,B,MKTG,5COP,JR,International,First,"[(pc, 0.4008), (worker, 0.3816), (task, 0.3673...","[(resume, 0.7073), (newbie, 0.3529), (think, 0...","[(resume, 0.7026), (helped, 0.5593)]","[(bsan, 1.0)]","[(stage, 0.4887), (grade, 0.438), (major, 0.38...","[(quick_learner, 0.6001), (sql, 0.4665), (exce...","[(really, 1.0)]","[(ethic, 0.437), (nhi, 0.4007), (determination...","[(nhi, 0.4235), (judgement, 0.3192), (ownershi...",[],"[(nhi, 0.501), (analyzing, 0.3596), (determina...",Spri
1379,5912,Spring/Summer,B,FIN,5COP,PJ,Domestic,First,"[(housing, 0.4449), (neighborhood, 0.376), (op...","[(student, 0.4284), (client, 0.3574), (backgro...","[(speech, 0.5285), (adjust, 0.2668), (project,...","[(client, 0.4369), (document, 0.4333), (due, 0...","[(client, 0.5682), (connecting, 0.4979), (coll...","[(analytical_skills, 0.9558), (want, 0.1806)]","[(housing, 0.7035), (year, 0.4392), (see, 0.20...","[(willie, 0.5105), (direction, 0.4055), (polit...","[(willie, 0.6511), (assertive, 0.4831), (littl...","[(investing, 0.6261), (willie, 0.4079), (chang...","[(willie, 0.5041), (polite, 0.38

In [3]:
dataset12 = pd.read_excel("ConsolidatedFiles/2017.xlsx")

In [4]:
dataset12

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,I wanted to get more hands on experience in ta...,More Excel classes and maybe see if we can get...,The classes i took that we used excel in helpe...,"Excel skills, along with basic tax knowledge.",I want to focus on the different tax rules and...,more knowledge on tax regulations.,Being in person at the office.,Asks good questions and positive attitude,na,na
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,Ever since I began pursuing a career in accoun...,I have no suggestions,I was able to apply what I learned in my accou...,All of my major accounting courses,I want to focus on excelling in my courses and...,I think I need to simply gain more knowledge i...,This job would have been much more interesting...,NaN,NaN,NaN
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,This co-op wasn't as beneficial as my first co...,NaN,I had to lead an Agile project and work with a...,Domestic and Global Outsourcing (MIS 347),NaN,Develop attainable goals and reasonable steps ...,The ability to interact with colleagues in office,Numbers based analytics / advanced excel,Self motivation (I believe this was a tough ye...,NaN
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,I learned how to troubleshoot Amazon's network...,NaN,"I learned on the job, and my desire to underst...",My coursework in MIS.,I want to focus on my MIS degree to pursue a c...,I need to develop my technical skills.,"If I had more female co-workers, the job would...",Christina is organized and focused. She accept...,There were times that she failed to get inform...,Just continue on the path of learning and take...
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,An aspect of this co-op experience that relate...,More accounting projects in classes that use M...,The financial accounting class I took at Drexe...,The coursework that prepared me for this co-op...,I hope to focus more on finance work in my ret...,Time Management_x000D_\nSetting goals and eval...,An onboarding process that included a comprehe...,Justin was able to pick up processes quickly a...,Justin has opportunity to develop his skills a...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,5562,Spring/Summer,B,MKTG,5COP,JR,International,First,This was my very first experience working at a...,I think it's better if we had more time for ou...,Helped with my resume.,"business, marketing and BSAN","I want to keep my grade in a good stage, and f...","excel, SQL and still quick learner","if I have more time for this, I really like th...",Nhi is very effective at analyzing data using ...,Nhi will need to continue to work on being mor...,None
1379,5912,Spring/Summer,B,FIN,5COP,PJ,Domestic,First,I have always cared about making sure there ar...,Make sure to inform students that clients may ...,Business communication projects forced me to a...,Mostly business communication due to the amoun...,I want to focus more on interpersonal communic...,I want to focus more on communication as well ...,Being able to see the housing process during t...,"Willie is polite and always willing to listen,...",Willie could be a little more assertive,"No changes recommended, our industry does not ..."
1380,5572,Spring/Summer,B,ACCT,5COP,JR,International,First,One of my professional goals is to become a CP...,"Making it more practical. For example, in acco...",I was prepared more by extracurricular activit...,Financial Reporting I & II,I'd like to take finance classes in addition t...,Procrastination and working at a better speed,Nothing that I can think of,Munira has been a fantastic asset to our Finan...,Munira is a great worker and therefore she has...,Introduce more real-life situational accountin...
1381,

In [5]:
sheetNames = dataset12.columns[8:]
columnList = list(dataset12.columns)
for i in range(8, len(columnList)):
    dataset12.rename({columnList[i]: sheetNames[i-8]}, axis=1, inplace=True)

In [6]:
datasetPP_1 = dataset12.copy()
# sheetNames.append('Skills_used')
# datasetPP_1['Skills_used'] = datasetPP_1['EMP-Strengths'] + datasetPP_1['EMP-Weaknesses']
datasetPP_1.head(50)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,I wanted to get more hands on experience in ta...,More Excel classes and maybe see if we can get...,The classes i took that we used excel in helpe...,"Excel skills, along with basic tax knowledge.",I want to focus on the different tax rules and...,more knowledge on tax regulations.,Being in person at the office.,Asks good questions and positive attitude,na,na,Asks good questions and positive attitudena
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,Ever since I began pursuing a career in accoun...,I have no suggestions,I was able to apply what I learned in my accou...,All of my major accounting courses,I want to focus on excelling in my courses and...,I think I need to simply gain more knowledge i...,This job would have been much more interesting...,NaN,NaN,NaN,NaN
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,This co-op wasn't as beneficial as my first co...,NaN,I had to lead an Agile project and work with a...,Domestic and Global Outsourcing (MIS 347),NaN,Develop attainable goals and reasonable steps ...,The ability to interact with colleagues in office,Numbers based analytics / advanced excel,Self motivation (I believe this was a tough ye...,NaN,Numbers based analytics / advanced excelSelf m...
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,I learned how to troubleshoot Amazon's network...,NaN,"I learned on the job, and my desire to underst...",My coursework in MIS.,I want to focus on my MIS degree to pursue a c...,I need to develop my technical skills.,"If I had more female co-workers, the job would...",Christina is organized and focused. She accept...,There were times that she failed to get inform...,Just continue on the path of learning and take...,Christina is organized and focused. She accept...
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,An aspect of this co-op experience that relate...,More accounting projects in classes that use M...,The financial accounting class I took at Drexe...,The coursework that prepared me for this co-op...,I hope to focus more on finance work in my ret...,Time Management_x000D_\nSetting goals and eval...,An onboarding process that included a comprehe...,Justin was able to pick up processes quickly a...,Justin has opportunity to develop his skills a...,NaN,Justin was able to pick up processes quickly a...
5,79,Fall/Winter,B,FIN,5COP,SR,Domestic,Third/Final,One professional goal that I am pursuing is to...,Have more classes like COM270 or MGMT450 to he...,Business communication prepared me for my co-o...,"I applied COM270, Business communication most ...","Last co-op, won't be returning to classes. Gra...",Data analytics,This job would have been more interesting if I...,NaN,NaN,NaN,NaN
6,2367,Spring/Summer,B,MKTG,5COP,SR,Domestic,Third/Final,"I have always been interested in events, and a...",n/a _x000D_\nIt's difficult to learn in the cl...,I was prepared for Co-Op from my previous 2 Co...,MKTG 367 - data-driven marketing_x000D_\nWhen ...,I added an IDM minor so I am excited to learn ...,Continuing to set professional boundaries.,"Being paid, not doing administrative tasks lik...","Alessandra had a wonderful attitude, was highl...",Alessandra was wonderful and an asset to our t...,None,"Alessandra had a wonderful attitude, was highl..."
7,2371,Spring/Summer,B,MKTG,5COP,SR,Domestic,Third/Final,"From the beginning of my time at Macquarie, I ...",Frame coursework like deliverables.,The higher-level classes I have taken prepared...,"Digital Data-Driven Marketing, Corporate Socia...",I would like to hone my skills in the areas I ...,I need to get more comfortable speaking in mee...,I would've like to have a deeper connection wi...,NaN,NaN,NaN,NaN
8,2374,Spring/Summer,B,MIS

In [8]:
def pre_process(text):
    text = str(text)
    new_words = []
    stop_words = []
    stop_words = set(stopwords.words('english'))
    
    with open("includeWordList.txt", 'r') as file:
        inclusiveWords = [line.strip() for line in file.readlines()]
    
#     text = includeWordsProcess(text, inclusiveWords)
    for substring in inclusiveWords:
        if len(substring.split()) == 2:
            replacement = '_'.join(substring.split())
            text = text.replace(substring, replacement)
#     return context
    
    ##Creating a list of custom stopwords from a file
    with open('stopWordList.txt') as f:
        new_words=[word for line in f for word in line.split()]
#     new_words = flat
    stop_words = list(stop_words.union(new_words))
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string - Tokenization
    text = text.split()
    
    # remove stopwords
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]
    
    #Stemming
#     for word in text:
#         print ("{0:20}{1:20}".format(word, ps.stem(word)))
    
    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    
#     return text    # returns list in the dataframe ['an', 'apple', 'a', 'day']
    strx = ' '.join(text) # returns list in the dataframe as a sentence/string (words are joined by empty space character) 'an apple a day'
    from keybert import KeyBERT
    kw_model = KeyBERT()
    # print("\nKeyBert Result\n")
    keywords = kw_model.extract_keywords(strx)
        #KeyBert Result
    kwResult = kw_model.extract_keywords(strx, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
#     res = str(kwResult).strip('[]')
    return kwResult




for i in range (0, len(sheetNames)):
    print(sheetNames[i])
    datasetPP_1[sheetNames[i]] = datasetPP_1[sheetNames[i]].apply(pre_process)

Goals
Classroom_activities_improve
Classroom_activities
Coursework
Return_to_class
Skills_to_develop
Job_interest
EMP-Strengths
EMP-Weaknesses
EMP-Feedback


In [9]:
datasetPP_1['Year'] = datasetPP_1['Work_Terms'].str.slice(0, 4) 
datasetPP_1.head(5)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used,Year
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,"[(senior, 0.3388), (planning, 0.3095), (return...","[(excel, 0.5205), (access, 0.5079), (class, 0....","[(excel, 0.4781), (flow, 0.3821), (class, 0.33...","[(excel, 0.6459), (skill, 0.5102), (along, 0.2...","[(cpa, 0.5948), (regulation, 0.5108), (exam, 0...","[(regulation, 1.0)]","[(person, 1.0)]","[(attitude, 0.5697), (asks, 0.5537), (question...",[],[],Asks good questions and positive attitudena,Spri
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,"[(accounting, 0.4684), (achieve, 0.3884), (pur...","[(suggestion, 1.0)]","[(accounting, 0.8058), (course, 0.2962)]","[(accounting, 0.8058), (course, 0.2962)]","[(excelling, 0.5827), (assignment, 0.4584), (a...","[(audit, 0.5346), (accounting, 0.5313), (emplo...","[(remotely, 0.4959), (control, 0.3852), (inter...","[(nan, 1.0)]","[(nan, 1.0)]","[(nan, 1.0)]",NaN,Fall
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,"[(consulting, 0.524), (agile, 0.4505), (remote...","[(nan, 1.0)]","[(agile, 0.4656), (collaborate, 0.4461), (lead...","[(outsourcing, 0.6944), (mi, 0.4187), (global,...","[(nan, 1.0)]","[(attainable, 0.5867), (goal, 0.5282), (reach,...","[(colleague, 0.6858), (interact, 0.6406)]","[(excel, 0.6599), (number, 0.4163), (advanced,...","[(motivation, 0.6571), (problem_solving, 0.493...","[(nan, 1.0)]",Numbers based analytics / advanced excelSelf m...,Fall
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,"[(cybersecurity, 0.4593), (vulnerability, 0.34...","[(nan, 1.0)]","[(experience, 0.4807), (hand, 0.4766), (desire...","[(mi, 1.0)]","[(mi, 0.5272), (degree, 0.4376), (graduation, ...","[(technical_skills, 0.9151), (need, 0.2497)]","[(worker, 0.6074), (female, 0.5425), (interest...","[(christina, 0.453), (personality, 0.4148), (t...","[(failed, 0.416), (key, 0.408), (growth, 0.334...","[(certification, 0.5955), (path, 0.5071), (fie...",Christina is organized and focused. She accept...,Spri
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,"[(networking, 0.486), (network, 0.4726), (meet...","[(excel, 0.3904), (accounting, 0.3644), (hour,...","[(accounting, 0.7399), (finance, 0.5188), (boo...","[(pivottables, 0.6948), (excel, 0.4705), (form...","[(accounting, 0.5042), (finance, 0.4552), (cla...","[(management_x, 0.4565), (progress, 0.416), (g...","[(onboarding, 0.5589), (email, 0.3812), (overv...","[(justin, 0.509), (communicates, 0.4013), (pic...","[(internship, 0.5264), (delay, 0.3652), (justi...","[(nan, 1.0)]",Justin was able to pick up processes quickly a...,Spri


In [ ]:
excel_filename = '2017PP.xlsx'
datasetPP_1.to_excel(excel_filename, index=False)
print(excel_filename)
datasetPP_1

In [3]:
dataset12 = pd.read_excel("ConsolidatedFiles/2018.xlsx")

In [4]:
dataset12

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,I wanted to get more hands on experience in ta...,More Excel classes and maybe see if we can get...,The classes i took that we used excel in helpe...,"Excel skills, along with basic tax knowledge.",I want to focus on the different tax rules and...,more knowledge on tax regulations.,Being in person at the office.,Asks good questions and positive attitude,na,na
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,Ever since I began pursuing a career in accoun...,I have no suggestions,I was able to apply what I learned in my accou...,All of my major accounting courses,I want to focus on excelling in my courses and...,I think I need to simply gain more knowledge i...,This job would have been much more interesting...,NaN,NaN,NaN
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,This co-op wasn't as beneficial as my first co...,NaN,I had to lead an Agile project and work with a...,Domestic and Global Outsourcing (MIS 347),NaN,Develop attainable goals and reasonable steps ...,The ability to interact with colleagues in office,Numbers based analytics / advanced excel,Self motivation (I believe this was a tough ye...,NaN
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,I learned how to troubleshoot Amazon's network...,NaN,"I learned on the job, and my desire to underst...",My coursework in MIS.,I want to focus on my MIS degree to pursue a c...,I need to develop my technical skills.,"If I had more female co-workers, the job would...",Christina is organized and focused. She accept...,There were times that she failed to get inform...,Just continue on the path of learning and take...
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,An aspect of this co-op experience that relate...,More accounting projects in classes that use M...,The financial accounting class I took at Drexe...,The coursework that prepared me for this co-op...,I hope to focus more on finance work in my ret...,Time Management_x000D_\nSetting goals and eval...,An onboarding process that included a comprehe...,Justin was able to pick up processes quickly a...,Justin has opportunity to develop his skills a...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,5562,Spring/Summer,B,MKTG,5COP,JR,International,First,This was my very first experience working at a...,I think it's better if we had more time for ou...,Helped with my resume.,"business, marketing and BSAN","I want to keep my grade in a good stage, and f...","excel, SQL and still quick learner","if I have more time for this, I really like th...",Nhi is very effective at analyzing data using ...,Nhi will need to continue to work on being mor...,None
1379,5912,Spring/Summer,B,FIN,5COP,PJ,Domestic,First,I have always cared about making sure there ar...,Make sure to inform students that clients may ...,Business communication projects forced me to a...,Mostly business communication due to the amoun...,I want to focus more on interpersonal communic...,I want to focus more on communication as well ...,Being able to see the housing process during t...,"Willie is polite and always willing to listen,...",Willie could be a little more assertive,"No changes recommended, our industry does not ..."
1380,5572,Spring/Summer,B,ACCT,5COP,JR,International,First,One of my professional goals is to become a CP...,"Making it more practical. For example, in acco...",I was prepared more by extracurricular activit...,Financial Reporting I & II,I'd like to take finance classes in addition t...,Procrastination and working at a better speed,Nothing that I can think of,Munira has been a fantastic asset to our Finan...,Munira is a great worker and therefore she has...,Introduce more real-life situational accountin...
1381,

In [5]:
sheetNames = dataset12.columns[8:]
columnList = list(dataset12.columns)
for i in range(8, len(columnList)):
    dataset12.rename({columnList[i]: sheetNames[i-8]}, axis=1, inplace=True)

In [6]:
datasetPP_1 = dataset12.copy()
# sheetNames.append('Skills_used')
# datasetPP_1['Skills_used'] = datasetPP_1['EMP-Strengths'] + datasetPP_1['EMP-Weaknesses']
datasetPP_1.head(50)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,I wanted to get more hands on experience in ta...,More Excel classes and maybe see if we can get...,The classes i took that we used excel in helpe...,"Excel skills, along with basic tax knowledge.",I want to focus on the different tax rules and...,more knowledge on tax regulations.,Being in person at the office.,Asks good questions and positive attitude,na,na,Asks good questions and positive attitudena
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,Ever since I began pursuing a career in accoun...,I have no suggestions,I was able to apply what I learned in my accou...,All of my major accounting courses,I want to focus on excelling in my courses and...,I think I need to simply gain more knowledge i...,This job would have been much more interesting...,NaN,NaN,NaN,NaN
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,This co-op wasn't as beneficial as my first co...,NaN,I had to lead an Agile project and work with a...,Domestic and Global Outsourcing (MIS 347),NaN,Develop attainable goals and reasonable steps ...,The ability to interact with colleagues in office,Numbers based analytics / advanced excel,Self motivation (I believe this was a tough ye...,NaN,Numbers based analytics / advanced excelSelf m...
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,I learned how to troubleshoot Amazon's network...,NaN,"I learned on the job, and my desire to underst...",My coursework in MIS.,I want to focus on my MIS degree to pursue a c...,I need to develop my technical skills.,"If I had more female co-workers, the job would...",Christina is organized and focused. She accept...,There were times that she failed to get inform...,Just continue on the path of learning and take...,Christina is organized and focused. She accept...
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,An aspect of this co-op experience that relate...,More accounting projects in classes that use M...,The financial accounting class I took at Drexe...,The coursework that prepared me for this co-op...,I hope to focus more on finance work in my ret...,Time Management_x000D_\nSetting goals and eval...,An onboarding process that included a comprehe...,Justin was able to pick up processes quickly a...,Justin has opportunity to develop his skills a...,NaN,Justin was able to pick up processes quickly a...
5,79,Fall/Winter,B,FIN,5COP,SR,Domestic,Third/Final,One professional goal that I am pursuing is to...,Have more classes like COM270 or MGMT450 to he...,Business communication prepared me for my co-o...,"I applied COM270, Business communication most ...","Last co-op, won't be returning to classes. Gra...",Data analytics,This job would have been more interesting if I...,NaN,NaN,NaN,NaN
6,2367,Spring/Summer,B,MKTG,5COP,SR,Domestic,Third/Final,"I have always been interested in events, and a...",n/a _x000D_\nIt's difficult to learn in the cl...,I was prepared for Co-Op from my previous 2 Co...,MKTG 367 - data-driven marketing_x000D_\nWhen ...,I added an IDM minor so I am excited to learn ...,Continuing to set professional boundaries.,"Being paid, not doing administrative tasks lik...","Alessandra had a wonderful attitude, was highl...",Alessandra was wonderful and an asset to our t...,None,"Alessandra had a wonderful attitude, was highl..."
7,2371,Spring/Summer,B,MKTG,5COP,SR,Domestic,Third/Final,"From the beginning of my time at Macquarie, I ...",Frame coursework like deliverables.,The higher-level classes I have taken prepared...,"Digital Data-Driven Marketing, Corporate Socia...",I would like to hone my skills in the areas I ...,I need to get more comfortable speaking in mee...,I would've like to have a deeper connection wi...,NaN,NaN,NaN,NaN
8,2374,Spring/Summer,B,MIS

In [8]:
def pre_process(text):
    text = str(text)
    new_words = []
    stop_words = []
    stop_words = set(stopwords.words('english'))
    
    with open("includeWordList.txt", 'r') as file:
        inclusiveWords = [line.strip() for line in file.readlines()]
    
#     text = includeWordsProcess(text, inclusiveWords)
    for substring in inclusiveWords:
        if len(substring.split()) == 2:
            replacement = '_'.join(substring.split())
            text = text.replace(substring, replacement)
#     return context
    
    ##Creating a list of custom stopwords from a file
    with open('stopWordList.txt') as f:
        new_words=[word for line in f for word in line.split()]
#     new_words = flat
    stop_words = list(stop_words.union(new_words))
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string - Tokenization
    text = text.split()
    
    # remove stopwords
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]
    
    #Stemming
#     for word in text:
#         print ("{0:20}{1:20}".format(word, ps.stem(word)))
    
    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    
#     return text    # returns list in the dataframe ['an', 'apple', 'a', 'day']
    strx = ' '.join(text) # returns list in the dataframe as a sentence/string (words are joined by empty space character) 'an apple a day'
    from keybert import KeyBERT
    kw_model = KeyBERT()
    # print("\nKeyBert Result\n")
    keywords = kw_model.extract_keywords(strx)
        #KeyBert Result
    kwResult = kw_model.extract_keywords(strx, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
#     res = str(kwResult).strip('[]')
    return kwResult




for i in range (0, len(sheetNames)):
    print(sheetNames[i])
    datasetPP_1[sheetNames[i]] = datasetPP_1[sheetNames[i]].apply(pre_process)

Goals
Classroom_activities_improve
Classroom_activities
Coursework
Return_to_class
Skills_to_develop
Job_interest
EMP-Strengths
EMP-Weaknesses
EMP-Feedback


In [9]:
datasetPP_1['Year'] = datasetPP_1['Work_Terms'].str.slice(0, 4) 
datasetPP_1.head(5)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used,Year
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,"[(senior, 0.3388), (planning, 0.3095), (return...","[(excel, 0.5205), (access, 0.5079), (class, 0....","[(excel, 0.4781), (flow, 0.3821), (class, 0.33...","[(excel, 0.6459), (skill, 0.5102), (along, 0.2...","[(cpa, 0.5948), (regulation, 0.5108), (exam, 0...","[(regulation, 1.0)]","[(person, 1.0)]","[(attitude, 0.5697), (asks, 0.5537), (question...",[],[],Asks good questions and positive attitudena,Spri
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,"[(accounting, 0.4684), (achieve, 0.3884), (pur...","[(suggestion, 1.0)]","[(accounting, 0.8058), (course, 0.2962)]","[(accounting, 0.8058), (course, 0.2962)]","[(excelling, 0.5827), (assignment, 0.4584), (a...","[(audit, 0.5346), (accounting, 0.5313), (emplo...","[(remotely, 0.4959), (control, 0.3852), (inter...","[(nan, 1.0)]","[(nan, 1.0)]","[(nan, 1.0)]",NaN,Fall
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,"[(consulting, 0.524), (agile, 0.4505), (remote...","[(nan, 1.0)]","[(agile, 0.4656), (collaborate, 0.4461), (lead...","[(outsourcing, 0.6944), (mi, 0.4187), (global,...","[(nan, 1.0)]","[(attainable, 0.5867), (goal, 0.5282), (reach,...","[(colleague, 0.6858), (interact, 0.6406)]","[(excel, 0.6599), (number, 0.4163), (advanced,...","[(motivation, 0.6571), (problem_solving, 0.493...","[(nan, 1.0)]",Numbers based analytics / advanced excelSelf m...,Fall
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,"[(cybersecurity, 0.4593), (vulnerability, 0.34...","[(nan, 1.0)]","[(experience, 0.4807), (hand, 0.4766), (desire...","[(mi, 1.0)]","[(mi, 0.5272), (degree, 0.4376), (graduation, ...","[(technical_skills, 0.9151), (need, 0.2497)]","[(worker, 0.6074), (female, 0.5425), (interest...","[(christina, 0.453), (personality, 0.4148), (t...","[(failed, 0.416), (key, 0.408), (growth, 0.334...","[(certification, 0.5955), (path, 0.5071), (fie...",Christina is organized and focused. She accept...,Spri
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,"[(networking, 0.486), (network, 0.4726), (meet...","[(excel, 0.3904), (accounting, 0.3644), (hour,...","[(accounting, 0.7399), (finance, 0.5188), (boo...","[(pivottables, 0.6948), (excel, 0.4705), (form...","[(accounting, 0.5042), (finance, 0.4552), (cla...","[(management_x, 0.4565), (progress, 0.416), (g...","[(onboarding, 0.5589), (email, 0.3812), (overv...","[(justin, 0.509), (communicates, 0.4013), (pic...","[(internship, 0.5264), (delay, 0.3652), (justi...","[(nan, 1.0)]",Justin was able to pick up processes quickly a...,Spri


In [ ]:
excel_filename = '2018PP.xlsx'
datasetPP_1.to_excel(excel_filename, index=False)
print(excel_filename)
datasetPP_1

In [3]:
dataset12 = pd.read_excel("ConsolidatedFiles/2019.xlsx")

In [4]:
dataset12

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,I wanted to get more hands on experience in ta...,More Excel classes and maybe see if we can get...,The classes i took that we used excel in helpe...,"Excel skills, along with basic tax knowledge.",I want to focus on the different tax rules and...,more knowledge on tax regulations.,Being in person at the office.,Asks good questions and positive attitude,na,na
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,Ever since I began pursuing a career in accoun...,I have no suggestions,I was able to apply what I learned in my accou...,All of my major accounting courses,I want to focus on excelling in my courses and...,I think I need to simply gain more knowledge i...,This job would have been much more interesting...,NaN,NaN,NaN
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,This co-op wasn't as beneficial as my first co...,NaN,I had to lead an Agile project and work with a...,Domestic and Global Outsourcing (MIS 347),NaN,Develop attainable goals and reasonable steps ...,The ability to interact with colleagues in office,Numbers based analytics / advanced excel,Self motivation (I believe this was a tough ye...,NaN
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,I learned how to troubleshoot Amazon's network...,NaN,"I learned on the job, and my desire to underst...",My coursework in MIS.,I want to focus on my MIS degree to pursue a c...,I need to develop my technical skills.,"If I had more female co-workers, the job would...",Christina is organized and focused. She accept...,There were times that she failed to get inform...,Just continue on the path of learning and take...
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,An aspect of this co-op experience that relate...,More accounting projects in classes that use M...,The financial accounting class I took at Drexe...,The coursework that prepared me for this co-op...,I hope to focus more on finance work in my ret...,Time Management_x000D_\nSetting goals and eval...,An onboarding process that included a comprehe...,Justin was able to pick up processes quickly a...,Justin has opportunity to develop his skills a...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,5562,Spring/Summer,B,MKTG,5COP,JR,International,First,This was my very first experience working at a...,I think it's better if we had more time for ou...,Helped with my resume.,"business, marketing and BSAN","I want to keep my grade in a good stage, and f...","excel, SQL and still quick learner","if I have more time for this, I really like th...",Nhi is very effective at analyzing data using ...,Nhi will need to continue to work on being mor...,None
1379,5912,Spring/Summer,B,FIN,5COP,PJ,Domestic,First,I have always cared about making sure there ar...,Make sure to inform students that clients may ...,Business communication projects forced me to a...,Mostly business communication due to the amoun...,I want to focus more on interpersonal communic...,I want to focus more on communication as well ...,Being able to see the housing process during t...,"Willie is polite and always willing to listen,...",Willie could be a little more assertive,"No changes recommended, our industry does not ..."
1380,5572,Spring/Summer,B,ACCT,5COP,JR,International,First,One of my professional goals is to become a CP...,"Making it more practical. For example, in acco...",I was prepared more by extracurricular activit...,Financial Reporting I & II,I'd like to take finance classes in addition t...,Procrastination and working at a better speed,Nothing that I can think of,Munira has been a fantastic asset to our Finan...,Munira is a great worker and therefore she has...,Introduce more real-life situational accountin...
1381,

In [5]:
sheetNames = dataset12.columns[8:]
columnList = list(dataset12.columns)
for i in range(8, len(columnList)):
    dataset12.rename({columnList[i]: sheetNames[i-8]}, axis=1, inplace=True)

In [6]:
datasetPP_1 = dataset12.copy()
# sheetNames.append('Skills_used')
# datasetPP_1['Skills_used'] = datasetPP_1['EMP-Strengths'] + datasetPP_1['EMP-Weaknesses']
datasetPP_1.head(50)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,I wanted to get more hands on experience in ta...,More Excel classes and maybe see if we can get...,The classes i took that we used excel in helpe...,"Excel skills, along with basic tax knowledge.",I want to focus on the different tax rules and...,more knowledge on tax regulations.,Being in person at the office.,Asks good questions and positive attitude,na,na,Asks good questions and positive attitudena
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,Ever since I began pursuing a career in accoun...,I have no suggestions,I was able to apply what I learned in my accou...,All of my major accounting courses,I want to focus on excelling in my courses and...,I think I need to simply gain more knowledge i...,This job would have been much more interesting...,NaN,NaN,NaN,NaN
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,This co-op wasn't as beneficial as my first co...,NaN,I had to lead an Agile project and work with a...,Domestic and Global Outsourcing (MIS 347),NaN,Develop attainable goals and reasonable steps ...,The ability to interact with colleagues in office,Numbers based analytics / advanced excel,Self motivation (I believe this was a tough ye...,NaN,Numbers based analytics / advanced excelSelf m...
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,I learned how to troubleshoot Amazon's network...,NaN,"I learned on the job, and my desire to underst...",My coursework in MIS.,I want to focus on my MIS degree to pursue a c...,I need to develop my technical skills.,"If I had more female co-workers, the job would...",Christina is organized and focused. She accept...,There were times that she failed to get inform...,Just continue on the path of learning and take...,Christina is organized and focused. She accept...
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,An aspect of this co-op experience that relate...,More accounting projects in classes that use M...,The financial accounting class I took at Drexe...,The coursework that prepared me for this co-op...,I hope to focus more on finance work in my ret...,Time Management_x000D_\nSetting goals and eval...,An onboarding process that included a comprehe...,Justin was able to pick up processes quickly a...,Justin has opportunity to develop his skills a...,NaN,Justin was able to pick up processes quickly a...
5,79,Fall/Winter,B,FIN,5COP,SR,Domestic,Third/Final,One professional goal that I am pursuing is to...,Have more classes like COM270 or MGMT450 to he...,Business communication prepared me for my co-o...,"I applied COM270, Business communication most ...","Last co-op, won't be returning to classes. Gra...",Data analytics,This job would have been more interesting if I...,NaN,NaN,NaN,NaN
6,2367,Spring/Summer,B,MKTG,5COP,SR,Domestic,Third/Final,"I have always been interested in events, and a...",n/a _x000D_\nIt's difficult to learn in the cl...,I was prepared for Co-Op from my previous 2 Co...,MKTG 367 - data-driven marketing_x000D_\nWhen ...,I added an IDM minor so I am excited to learn ...,Continuing to set professional boundaries.,"Being paid, not doing administrative tasks lik...","Alessandra had a wonderful attitude, was highl...",Alessandra was wonderful and an asset to our t...,None,"Alessandra had a wonderful attitude, was highl..."
7,2371,Spring/Summer,B,MKTG,5COP,SR,Domestic,Third/Final,"From the beginning of my time at Macquarie, I ...",Frame coursework like deliverables.,The higher-level classes I have taken prepared...,"Digital Data-Driven Marketing, Corporate Socia...",I would like to hone my skills in the areas I ...,I need to get more comfortable speaking in mee...,I would've like to have a deeper connection wi...,NaN,NaN,NaN,NaN
8,2374,Spring/Summer,B,MIS

In [8]:
def pre_process(text):
    text = str(text)
    new_words = []
    stop_words = []
    stop_words = set(stopwords.words('english'))
    
    with open("includeWordList.txt", 'r') as file:
        inclusiveWords = [line.strip() for line in file.readlines()]
    
#     text = includeWordsProcess(text, inclusiveWords)
    for substring in inclusiveWords:
        if len(substring.split()) == 2:
            replacement = '_'.join(substring.split())
            text = text.replace(substring, replacement)
#     return context
    
    ##Creating a list of custom stopwords from a file
    with open('stopWordList.txt') as f:
        new_words=[word for line in f for word in line.split()]
#     new_words = flat
    stop_words = list(stop_words.union(new_words))
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string - Tokenization
    text = text.split()
    
    # remove stopwords
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]
    
    #Stemming
#     for word in text:
#         print ("{0:20}{1:20}".format(word, ps.stem(word)))
    
    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    
#     return text    # returns list in the dataframe ['an', 'apple', 'a', 'day']
    strx = ' '.join(text) # returns list in the dataframe as a sentence/string (words are joined by empty space character) 'an apple a day'
    from keybert import KeyBERT
    kw_model = KeyBERT()
    # print("\nKeyBert Result\n")
    keywords = kw_model.extract_keywords(strx)
        #KeyBert Result
    kwResult = kw_model.extract_keywords(strx, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
#     res = str(kwResult).strip('[]')
    return kwResult




for i in range (0, len(sheetNames)):
    print(sheetNames[i])
    datasetPP_1[sheetNames[i]] = datasetPP_1[sheetNames[i]].apply(pre_process)

Goals
Classroom_activities_improve
Classroom_activities
Coursework
Return_to_class
Skills_to_develop
Job_interest
EMP-Strengths
EMP-Weaknesses
EMP-Feedback


In [9]:
datasetPP_1['Year'] = datasetPP_1['Work_Terms'].str.slice(0, 4) 
datasetPP_1.head(5)

,Responder_ID,Work_Terms,College,Major,Conc,Class,Citizenship_Status,Co-op_#,Goals,Classroom_activities_improve,Classroom_activities,Coursework,Return_to_class,Skills_to_develop,Job_interest,EMP-Strengths,EMP-Weaknesses,EMP-Feedback,Skills_used,Year
0,2181,Spring/Summer,B,ACCT,5COP,SR,Domestic,Third/Final,"[(senior, 0.3388), (planning, 0.3095), (return...","[(excel, 0.5205), (access, 0.5079), (class, 0....","[(excel, 0.4781), (flow, 0.3821), (class, 0.33...","[(excel, 0.6459), (skill, 0.5102), (along, 0.2...","[(cpa, 0.5948), (regulation, 0.5108), (exam, 0...","[(regulation, 1.0)]","[(person, 1.0)]","[(attitude, 0.5697), (asks, 0.5537), (question...",[],[],Asks good questions and positive attitudena,Spri
1,72,Fall/Winter,B,ACCT,5COP,SR,Domestic,Third/Final,"[(accounting, 0.4684), (achieve, 0.3884), (pur...","[(suggestion, 1.0)]","[(accounting, 0.8058), (course, 0.2962)]","[(accounting, 0.8058), (course, 0.2962)]","[(excelling, 0.5827), (assignment, 0.4584), (a...","[(audit, 0.5346), (accounting, 0.5313), (emplo...","[(remotely, 0.4959), (control, 0.3852), (inter...","[(nan, 1.0)]","[(nan, 1.0)]","[(nan, 1.0)]",NaN,Fall
2,74,Fall/Winter,B,MIS,5COP,SR,Domestic,Third/Final,"[(consulting, 0.524), (agile, 0.4505), (remote...","[(nan, 1.0)]","[(agile, 0.4656), (collaborate, 0.4461), (lead...","[(outsourcing, 0.6944), (mi, 0.4187), (global,...","[(nan, 1.0)]","[(attainable, 0.5867), (goal, 0.5282), (reach,...","[(colleague, 0.6858), (interact, 0.6406)]","[(excel, 0.6599), (number, 0.4163), (advanced,...","[(motivation, 0.6571), (problem_solving, 0.493...","[(nan, 1.0)]",Numbers based analytics / advanced excelSelf m...,Fall
3,5586,Spring/Summer,B,FIN,5COP,SR,Domestic,Second,"[(cybersecurity, 0.4593), (vulnerability, 0.34...","[(nan, 1.0)]","[(experience, 0.4807), (hand, 0.4766), (desire...","[(mi, 1.0)]","[(mi, 0.5272), (degree, 0.4376), (graduation, ...","[(technical_skills, 0.9151), (need, 0.2497)]","[(worker, 0.6074), (female, 0.5425), (interest...","[(christina, 0.453), (personality, 0.4148), (t...","[(failed, 0.416), (key, 0.408), (growth, 0.334...","[(certification, 0.5955), (path, 0.5071), (fie...",Christina is organized and focused. She accept...,Spri
4,2366,Spring/Summer,B,FIN,5COP,SR,Domestic,Third/Final,"[(networking, 0.486), (network, 0.4726), (meet...","[(excel, 0.3904), (accounting, 0.3644), (hour,...","[(accounting, 0.7399), (finance, 0.5188), (boo...","[(pivottables, 0.6948), (excel, 0.4705), (form...","[(accounting, 0.5042), (finance, 0.4552), (cla...","[(management_x, 0.4565), (progress, 0.416), (g...","[(onboarding, 0.5589), (email, 0.3812), (overv...","[(justin, 0.509), (communicates, 0.4013), (pic...","[(internship, 0.5264), (delay, 0.3652), (justi...","[(nan, 1.0)]",Justin was able to pick up processes quickly a...,Spri


In [ ]:
excel_filename = '2019PP.xlsx'
datasetPP_1.to_excel(excel_filename, index=False)
print(excel_filename)
datasetPP_1

In [15]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

In [16]:
def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(text)['compound']
    return sentiment_score

In [13]:
import pandas as pd
fp = "ConsolidatedFiles/"
df2017 = pd.read_excel(fp+"2017.xlsx")
df2018 = pd.read_excel(fp+'2018.xlsx', 'Sheet1')
df2019 = pd.read_excel(fp+'2019.xlsx')
df2020 = pd.read_excel(fp+'2020_1.xlsx', 'Sheet1')

In [17]:
df2017['EMP-Feedback'] = df2017['EMP-Feedback'].astype(str)
df2017['Sentiment'] = df2017['EMP-Feedback'].apply(get_sentiment)
df2017

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - '/Users/shubhamarundekatey/nltk_data'
    - '/usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/nltk_data'
    - '/usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/share/nltk_data'
    - '/usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
df2018['EMP_FEEDBACK'] = df2018['EMP_FEEDBACK'].astype(str)
df2018['Sentiment'] = df2018['EMP_FEEDBACK'].apply(get_sentiment)

df2018

In [ ]:
df2019['EMP_Feedback'] = df2019['EMP_Feedback'].astype(str)
df2019['Sentiment'] = df2019['EMP_Feedback'].apply(get_sentiment)
df2019

In [ ]:
df2020['EMP-Feedback'] = df2020['EMP-Feedback'].astype(str)
df2020['Sentiment'] = df2020['EMP-Feedback'].apply(get_sentiment)
df2020

In [ ]:
alldf = [df2017, df2018, df2019, df2020]
sa2017 = pd.DataFrame()
sa2018 = pd.DataFrame()
sa2019 = pd.DataFrame()
sa2020 = pd.DataFrame()
newdf = [sa2017, sa2018, sa2019, sa2020]
# selectedCols = ['Responder ID', 'Terms', 'College', 'Major', 'Conc', 'Co-op #', 'Citizenship', 'Sentiment']
for i in range(0, len(alldf)):
    selectedCols = list(alldf[i].columns[:8])
    selectedCols.append('Sentiment')
    print(selectedCols)
    newdf[i] = pd.DataFrame(alldf[i][selectedCols])

In [ ]:
sa2017.to_excel('sa2017.xlsx')
sa2018.to_excel('sa2018.xlsx')
sa2019.to_excel('sa2019.xlsx')
sa2020.to_excel('sa2020.xlsx')